In [30]:
"""Converting mesh objects into simplified skeletons and using skeleton indices to 
create cutouts of EM and segmentation data to input to the myelin detection model"""

In [ ]:
from cloudvolume import CloudVolume
from meshparty import trimesh_io
import numpy as np
from caveclient import CAVEclient
from pcg_skel import pcg_meshwork
import pcg_skel
import imageryclient as ic
from PIL import Image
from meshparty import trimesh_io

In [17]:
client = CAVEclient('minnie65_public_v117')

In [18]:
client.info.get_datastack_info()

{'synapse_table': 'synapses_pni_2',
 'local_server': 'https://minnie.microns-daf.com',
 'description': 'The publicly available version of the minnie65 volume.',
 'segmentation_source': 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public_v117',
 'analysis_database': None,
 'viewer_site': 'https://neuromancer-seung-import.appspot.com/',
 'aligned_volume': {'name': 'minnie65_phase3',
  'id': 1,
  'image_source': 'precomputed://https://bossdb-open-data.s3.amazonaws.com/microns/minnie/minnie65-phase3-em/aligned/v1',
  'description': "This is the second alignment of the IARPA 'minnie65' dataset, completed in the spring of 2020 that used the seamless approach."},
 'soma_table': 'nucleus_neuron_svm'}

In [19]:
mm = trimesh_io.MeshMeta(cv_path=client.info.segmentation_source(),
                         disk_cache_path='minniemeshes')

In [20]:
#imagery client
img_src = 'precomputed://https://bossdb-open-data.s3.amazonaws.com/microns/minnie/minnie65-phase3-em/aligned/v1'
seg_src = client.info.segmentation_source()

#'precomputed://gs://iarpa_microns/minnie/minnie65/seg'
img_client = ic.ImageryClient(image_source=img_src,
                              segmentation_source=seg_src)

In [21]:
cldvol = CloudVolume(seg_src, mip = [8, 8, 40], progress = False, use_https = True, parallel=24)
cldvol2 = CloudVolume(img_src, progress = False, use_https = True, parallel=24)

In [22]:
#seg_ids
idList = ["864691135467660940",
          "864691135212863360",
          "864691135699372834",
        "864691135730415801",
        "864691134940133219",
          "864691135785592004", "864691136118647832", "864691135776002477", "864691135837451155","864691135462540445", "864691135994684074"]

In [23]:
#generate a skeleton
def getSkeleton(root_id, client, cldvol):
    return pcg_meshwork(root_id = root_id, client = client, cv = cldvol, refine = 'all')

In [24]:
#making a directory to save imgs and segs
import os

if !os.path.isdir("imgsEM"):
    os.mkdir("imgsEM")
if !os.path.isdir("segmentations"):
    os.mkdir("segmentations")
#img path
img_path = os.getcwd() + "\\imgsEM\\"
#seg path
seg_path = os.getcwd() + "\\segmentations\\"

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'imgsEM'

In [25]:
#make dirs per root id
import os
for id_ in idList:
    if !os.path.isdir(img_path+id_):
        os.mkdir(img_path+id_)
    if !os.path.isdir(seg_path+id_):
        os.mkdir(seg_path+id_)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sanah.imani\\MyelinProj\\imgsEM\\\\864691135467660940\\'

In [26]:
#save cutouts
def save_cutouts(cutouts,index,id_):
    img = cutouts[0]
    seg = cutouts[1]
    np.save(img_path+id_+"\\"+"img"+str(index), img.T)
    np.save(seg_path+id_+"\\"+"seg"+str(index),seg.T)

In [ ]:
#visualise cutouts
def visualise_cutouts(path):
    im = Image.fromarray(path)
    

In [27]:
#generate cutouts
def getAndSave_cutouts(skel, vertices, dims,img_client,id_):
    for i in range(len(vertices)):
        ctr = []
        width = height = depth = 3000
        bounds = ic.bounds_from_center(ctr, width=width, height=height, depth=depth)
        image = img_client.image_cutout(bounds)
        seg = img_client.segmentation_cutout(bounds)
        save_cutouts([image,seg],i,id_)
#https://github.com/AllenInstitute/ImageryClient/blob/master/imageryclient/imagery.py#L81
# 3 microns

In [28]:
#main 
def main(id_,cldvol, CVclient,img_client):
    #mesh = mm.mesh(seg_id = int(id_))
    skel = getSkeleton(int(id_), CVclient, cldvol)
    #nanometers 
    vertices = skel.vertices
    dims = pcg_skel.util.chunk_dims(cldvol)
    getAndSave_cutouts(skel,vertices, dims,img_client,id_)

In [31]:
#run
for id_ in idList:
    main(id_,cldvol, client, img_client)
    

100%|██████████| 18182/18182 [00:05<00:00, 3048.81it/s]


AttributeError: module 'pcg_skel' has no attribute 'util'

In [ ]:
#reading from h5 files
def readh5file():
    